In [ ]:
import argparse
import csv
import os
import re
import time
from typing import List, Dict

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd


In [2]:


def scrape_homedy(url):
    """
    Scrapes real estate data from Homedy.com and returns a list of dictionaries.
    Includes robust error logging.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        print(f"Đang gửi yêu cầu đến URL: {url}")
        response = requests.get(url, headers=headers)
        print(f"Mã trạng thái HTTP nhận được: {response.status_code}")
        response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)

        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Check for the product container class
        product_items = soup.find_all('div', class_='product-item')
        
        if not product_items:
            print("Lỗi: Không tìm thấy phần tử nào với class 'product-item'.")
            print("Có thể cấu trúc HTML của trang web đã thay đổi.")
            return []

        data_list = []
        for item in product_items:
            try:
                # Scrape title
                title_tag = item.find('h3', class_='product-title')
                title = title_tag.get_text(strip=True) if title_tag else None

                # Scrape price
                price_tag = item.find('span', class_='price')
                price = price_tag.get_text(strip=True) if price_tag else None
                
                # Scrape location
                location_tag = item.find('span', class_='address-text')
                location = location_tag.get_text(strip=True) if location_tag else None
                
                # Scrape area
                area_tag = item.find('span', class_='area')
                area = area_tag.get_text(strip=True) if area_tag else None

                if title:
                    data_list.append({
                        'title': title,
                        'price': price,
                        'location': location,
                        'area': area
                    })
            except Exception as e:
                print(f"Lỗi khi xử lý một sản phẩm: {e}")
                continue

        return data_list

    except requests.exceptions.RequestException as e:
        print(f"Lỗi trong quá trình kết nối hoặc yêu cầu: {e}")
        return None

if __name__ == "__main__":
    target_url = "https://homedy.com/ban-nha-dat-ha-noi"
    scraped_data = scrape_homedy(target_url)

    if scraped_data:
        print(f"\n--- Thu thập thành công {len(scraped_data)} sản phẩm: ---")
        for i, item in enumerate(scraped_data[:3], 1):  # Print first 3 items for a quick check
            print(f"\nSản phẩm {i}:")
            for key, value in item.items():
                print(f"  - {key.capitalize()}: {value}")
    else:
        print("\nKhông thể thu thập dữ liệu.")

Đang gửi yêu cầu đến URL: https://homedy.com/ban-nha-dat-ha-noi
Mã trạng thái HTTP nhận được: 200

Không thể thu thập dữ liệu.
